# Problem 2

In [1]:
%matplotlib inline
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import math
import numpy as np
import pandas as pd
import pathlib
import six
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
pd.set_option("display.width", 1000)
pd.set_option("display.max_rows", 50)
plt.style.use("ggplot")
plt.rcParams["font.size"] = 13
mpl.rcParams["font.family"] = "Osaka"

## Data

In [2]:
data_path = pathlib.Path("./DataPS201901.csv")

In [3]:
df = pd.read_csv(data_path, header=None, names=["choice", "age", "gender"])
df

choice  age  gender
0          3   46       1
1          0   29       1
2          1   27       0
3          3   36       0
4          3   39       0
5          3   54       1
6          3   42       0
7          0   23       1
8          2   35       0
9          3   58       0
10         3   37       1
11         3   35       1
12         3   55       0
13         3   52       0
14         2   35       0
15         3   52       1
16         0   22       1
17         3   44       0
18         0   27       1
19         3   58       1
20         0   29       1
21         3   37       0
22         3   43       0
23         1   37       0
24         3   31       0
...      ...  ...     ...
9975       2   45       0
9976       2   51       0
9977       3   36       0
9978       1   47       0
9979       2   31       0
9980       3   44       1
9981       3   36       0
9982       3   50       1
9983       3   34       1
9984       3   29       1
9985       0   38       1
9986       0   32       1
9987       0   38       1
9988       3   34       0
9989       3   42       0
9990       3   60       0
9991       0   24       0
9992       3   23       0
9993       3   34       0
9994       0   34       1
9995       3   40       1
9996       0   38       1
9997       3   56       0
9998       3   34       0
9999       3   32       1

[10000 rows x 3 columns]

In [4]:
car_spec = np.array([[0, 0], [1, 5], [1.2, 3.5], [1.4, 2]])

## Estimation

The utility of player $i$ when choosing car $j$ is given by
\begin{align}
u_{ij} = \begin{cases}
\beta_0 + \beta_1 \cdot age_i \cdot hp_j + \beta_2 \cdot gender_i \cdot fe_j + \epsilon_{ij} & \text{if $j \neq 0$}\\
0 & \text{if $j = 0$}
\end{cases}
\end{align}
where $\epsilon_{ij} \overset{\text{i.i.d.}}{\sim} F$, $F$ is type I extreme distribution.


Then the log likelihood function is given by
\begin{align}
\ell(\beta) \equiv \sum_i \sum_j y_{ij} \log P_{ij} = \sum_i \sum_j y_{ij} \left( \beta^\text{T} x_{ij} - \log \left(\sum_{k=0}^3 \exp(\beta^\text{T} x_{ik}) \right) \right)
\end{align}
where $P_{ij} \equiv \exp(\beta^\text{T} x_{ij}) / \sum_{k=0}^3 \exp(\beta^\text{T} x_{ik})$, $x_{i0} = (0, 0, 0)^\text{T}$, and $x_{ij} = (1, age_i \cdot hp_j, gender_i \cdot fe_j)^\text{T}$ for $j > 0$.

Since $\ell(\beta)$ is concave in $\beta$ (I will show the concavity in the last page), I can obtain the global maximum by the simple Newton method.

In [5]:
# choice, agent, beta_index
covariates = np.empty([4, df.shape[0], 3], dtype=float)
covariates[:, :, [1, 2]] = car_spec[:, None, :]
covariates[:, :, 0] = 1
covariates[0, :, 0] = 0
covariates[:, :, 1] *= df["age"][None, :]
covariates[:, :, 2] *= df["gender"][None, :]
covariates

array([[[ 0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ],
        ...,
        [ 0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ]],

       [[ 1. , 46. ,  5. ],
        [ 1. , 29. ,  5. ],
        [ 1. , 27. ,  0. ],
        ...,
        [ 1. , 56. ,  0. ],
        [ 1. , 34. ,  0. ],
        [ 1. , 32. ,  5. ]],

       [[ 1. , 55.2,  3.5],
        [ 1. , 34.8,  3.5],
        [ 1. , 32.4,  0. ],
        ...,
        [ 1. , 67.2,  0. ],
        [ 1. , 40.8,  0. ],
        [ 1. , 38.4,  3.5]],

       [[ 1. , 64.4,  2. ],
        [ 1. , 40.6,  2. ],
        [ 1. , 37.8,  0. ],
        ...,
        [ 1. , 78.4,  0. ],
        [ 1. , 47.6,  0. ],
        [ 1. , 44.8,  2. ]]])

In [6]:
y = np.identity(4)[df["choice"]].T

def obj_fun(beta):
    beta_x = covariates @ beta
    inside_par = beta_x - np.log(np.sum(np.exp(beta_x), axis=0))
    return -1 * np.sum(y * inside_par, axis=None)


In [7]:
beta_init = np.array([1.0, 1.0, 1.0])
result = opt.minimize(obj_fun, beta_init, method="BFGS")
result

      fun: 8361.76973953566
 hess_inv: array([[ 0.,  0., -0.],
       [ 0.,  0., -0.],
       [-0., -0.,  0.]])
      jac: array([0.   , 0.001, 0.   ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 166
      nit: 21
     njev: 33
   status: 2
  success: False
        x: array([-1.887,  0.097, -1.027])

The estimates are $(\hat{\beta}_0^{\text{MLE}}, \hat{\beta}_1^{\text{MLE}}, \hat{\beta}_2^{\text{MLE}}) = (-1.887, 0.097, -1.027)$